# LangChain Part 3: Output Parsing
- Important when using agents with tools e.g. calculator , search engine 
- Many systems expect data in specific way

In [1]:
from langchain_openai import ChatOpenAI
from langchain_ollama.chat_models import ChatOllama
from os import getenv
from dotenv import load_dotenv
from pprint import pprint

load_dotenv()

llm = ChatOpenAI(
    openai_api_key=getenv("OPENROUTER_API_KEY"),
    openai_api_base="https://openrouter.ai/api/v1",
    model="microsoft/phi-3-medium-128k-instruct:free",
    temperature=0.2,
    max_tokens=512
)
# llm = ChatOllama(model='llama3.2', temperature=0.2, max_tokens=512)

result = llm.invoke("Hello, how are you today?")
print(result.content)

 I'm just a computer program, so I don't have feelings, but I'm ready to assist you! How can I help you today?


# CSV Parser 
- Used when want to return a list of comma-separated Items

In [2]:
from functools import partial
from pprint import pp
from tempfile import template
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate


output_parser = CommaSeparatedListOutputParser()

format_instructions = output_parser.get_format_instructions()
print("Format Instructions:\n",format_instructions, "\n")

prompt = PromptTemplate(
    template="List five {subject}. \n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)

pprint(prompt.dict())

# Create a model that uses the prompt
chain = prompt | llm | output_parser

# Invoke the model
result = chain.invoke({"subject": "dog breeds"})

print('\nResult:\n',result)


Format Instructions:
 Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz` 

{'_type': 'prompt',
 'input_variables': ['subject'],
 'metadata': None,
 'name': None,
 'optional_variables': [],
 'output_parser': None,
 'partial_variables': {'format_instructions': 'Your response should be a list '
                                              'of comma separated values, eg: '
                                              '`foo, bar, baz` or '
                                              '`foo,bar,baz`'},
 'tags': None,
 'template': 'List five {subject}. \n{format_instructions}',
 'template_format': 'f-string',
 'validate_template': False}

Result:
 ['Beagle', 'Labrador Retriever', 'German Shepherd', 'Bulldog', 'Poodle']


# Datetime Parser
- Used when want to return a date in a specific format
- Useful for applications that require a specific date format like a calendar

In [5]:
from langchain.output_parsers import DatetimeOutputParser
from langchain_core.prompts import PromptTemplate

output_parser = DatetimeOutputParser()

# Create a new prompt template for datetime parsing
datetime_prompt = PromptTemplate(
    template="What is the current date and time in UTC?\n{format_instructions}",
    input_variables=["format_instructions"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)
pprint(datetime_prompt.dict())


{'_type': 'prompt',
 'input_variables': [],
 'metadata': None,
 'name': None,
 'optional_variables': [],
 'output_parser': None,
 'partial_variables': {'format_instructions': 'Write a datetime string that '
                                              'matches the following pattern: '
                                              "'%Y-%m-%dT%H:%M:%S.%fZ'.\n"
                                              '\n'
                                              'Examples: '
                                              '0142-03-27T06:31:47.571059Z, '
                                              '0496-03-28T23:36:36.585184Z, '
                                              '0078-08-30T22:40:22.919265Z\n'
                                              '\n'
                                              'Return ONLY this string, no '
                                              'other words!'},
 'tags': None,
 'template': 'What is the current date and time in UTC?\n{format_instructions}',
 'templ

In [7]:
# Create a chain with the new prompt and the existing llm and output_parser
datetime_chain = datetime_prompt | llm | output_parser

# Invoke the chain
datetime_result = datetime_chain.invoke({})

print('\nDatetime Result:\n', datetime_result)
print(type(datetime_result))


Datetime Result:
 2022-07-20 15:12:43.611809
<class 'datetime.datetime'>


# Enum Parser
- Used when want to return a list of items in a specific format
- e.g. list of colors, list of countries
- Useful for applications that require a specific list of items like a dropdown menu.

In [11]:
import re
from langchain.output_parsers.enum import EnumOutputParser
from langchain_core.prompts import PromptTemplate
from enum import Enum
from langchain_core.prompts import PromptTemplate

class EyeColor(Enum):
    RED = "Red"
    GREEN = "Green"
    BLUE = "Blue"
    BROWN = "Brown"
    HAZEL = "Hazel"
    AMBER = "Amber"


parser = EnumOutputParser(enum=EyeColor)




prompt = PromptTemplate.from_template(
    """What color eyes does this person have?

> Person: {person}

Instructions: {instructions}"""
).partial(instructions=parser.get_format_instructions())

print(parser.get_format_instructions())
pprint(prompt.dict())

Select one of the following options: Red, Green, Blue, Brown, Hazel, Amber
{'_type': 'prompt',
 'input_variables': ['person'],
 'metadata': None,
 'name': None,
 'optional_variables': [],
 'output_parser': None,
 'partial_variables': {'instructions': 'Select one of the following options: '
                                       'Red, Green, Blue, Brown, Hazel, Amber'},
 'tags': None,
 'template': 'What color eyes does this person have?\n'
             '\n'
             '> Person: {person}\n'
             '\n'
             'Instructions: {instructions}',
 'template_format': 'f-string',
 'validate_template': False}


In [12]:
chain = prompt | llm | parser

result = chain.invoke({"person": "This person has blue eyes."})

print(result)

EyeColor.BLUE


# Json Parser
- Returns data in JSON format 
- e.g. "{"name": "John", "age": 30, "city": "New York"}"
- Useful for applications that require data in JSON format like APIs, web services

In [15]:
from langchain_core.output_parsers import JsonOutputParser

# Create a JSON output parser
json_output_parser = JsonOutputParser()

# Define the format instructions
json_format_instructions = json_output_parser.get_format_instructions()
print("Format Instructions:\n", json_format_instructions, "\n")

# Create a prompt template for JSON parsing
json_prompt = PromptTemplate(
    template="Provide the details of a person in JSON format.\n{format_instructions}",
    input_variables=["format_instructions"],
    partial_variables={"format_instructions": json_format_instructions},
)

pprint(json_prompt.dict())


# Create a chain with the new prompt and the existing llm and json_output_parser
json_chain = json_prompt | llm | json_output_parser

# Invoke the chain
json_result = json_chain.invoke({})

print('\nReturn:\n',json_result,'\n')
pprint(json_result)

Format Instructions:
 Return a JSON object. 

{'_type': 'prompt',
 'input_variables': [],
 'metadata': None,
 'name': None,
 'optional_variables': [],
 'output_parser': None,
 'partial_variables': {'format_instructions': 'Return a JSON object.'},
 'tags': None,
 'template': 'Provide the details of a person in JSON format.\n'
             '{format_instructions}',
 'template_format': 'f-string',
 'validate_template': False}

Return:
 {'name': 'John Doe', 'age': 30, 'gender': 'Male', 'occupation': 'Software Engineer', 'address': {'street': '123 Main St', 'city': 'New York', 'state': 'NY', 'zip': '10001'}, 'phone': {'home': '555-1234', 'work': '555-5678'}, 'email': 'johndoe@example.com'} 

{'address': {'city': 'New York',
             'state': 'NY',
             'street': '123 Main St',
             'zip': '10001'},
 'age': 30,
 'email': 'johndoe@example.com',
 'gender': 'Male',
 'name': 'John Doe',
 'occupation': 'Software Engineer',
 'phone': {'home': '555-1234', 'work': '555-5678'}}


In [17]:
# Streaming 
for s in llm.stream("tell me a story"):
    print(s)


content='' additional_kwargs={} response_metadata={} id='run-fe1f8ce2-89be-4ac9-9b08-f72e358f0120'
content=' Once' additional_kwargs={} response_metadata={} id='run-fe1f8ce2-89be-4ac9-9b08-f72e358f0120'
content=' upon' additional_kwargs={} response_metadata={} id='run-fe1f8ce2-89be-4ac9-9b08-f72e358f0120'
content=' a' additional_kwargs={} response_metadata={} id='run-fe1f8ce2-89be-4ac9-9b08-f72e358f0120'
content=' time' additional_kwargs={} response_metadata={} id='run-fe1f8ce2-89be-4ac9-9b08-f72e358f0120'
content=',' additional_kwargs={} response_metadata={} id='run-fe1f8ce2-89be-4ac9-9b08-f72e358f0120'
content=' in' additional_kwargs={} response_metadata={} id='run-fe1f8ce2-89be-4ac9-9b08-f72e358f0120'
content=' a' additional_kwargs={} response_metadata={} id='run-fe1f8ce2-89be-4ac9-9b08-f72e358f0120'
content=' l' additional_kwargs={} response_metadata={} id='run-fe1f8ce2-89be-4ac9-9b08-f72e358f0120'
content='ush' additional_kwargs={} response_metadata={} id='run-fe1f8ce2-89be-4ac9-9

KeyboardInterrupt: 